In [2]:
def load_image(filename):
    image = Image.open(filename)
    image = np.
    image = np.clip(image, 0, 4095)
    return np.array(image)

def process_raw_image(image):
    mode = stats.mode(image, None)[0][0]
    vert = np.all(image[:, image.shape[1]//2-5:image.shape[1]//2+5] != mode, axis=1)
    horiz = np.all(image[image.shape[0]//2-5:image.shape[0]//2+5, :] != mode, axis=0)
    image = image[vert][:,horiz]
    return image
 
def is_inverted(image, corner_ratio=0.2, center_ratio=0.01, bottom=True):
    height, width = image.shape
    corner_size = (int(height*corner_ratio), int(width*corner_ratio))
    tl = image[:corner_size[0], :corner_size[1]].flatten()
    bl = image[-corner_size[0]:, :corner_size[1]].flatten()
    tr = image[:corner_size[0], -corner_size[1]:].flatten()
    br = image[-corner_size[0]:, -corner_size[1]:].flatten()
    if bottom:
        corners = np.hstack([tl, bl, tr, br])
    else:
        corners = np.hstack([tl, tr])
            
    center = image[
        int(height*(0.5-center_ratio)):int(height*(0.5+center_ratio)),
        int(width*(0.5-center_ratio)):int(width*(0.5+center_ratio))
    ]
    return center.mean() < corners.mean()